In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = [None] * 2  # Using 0-based indexing
node[0] = Delay(model, 'Delay') 
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
jobclass = [None] * 2  # Using 0-based indexing
jobclass[0] = ClosedClass(model, 'Class1', 3, node[1], 0)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[1], 0)
node[1].set_number_of_servers(3)

node[0].set_service(jobclass[0], Exp(1))
node[0].set_service(jobclass[1], Exp(1))
node[1].set_service(jobclass[0], Exp(1.2))
node[1].set_service(jobclass[1], Erlang.fit_mean_and_scv(1.0, 0.5))

M = model.get_number_of_stations()
K = model.get_number_of_classes()

P = model.init_routing_matrix()
# Class 1 to Class 1 routing matrix
P.set(jobclass[0], jobclass[0], node[0], node[0], 0.3)
P.set(jobclass[0], jobclass[0], node[0], node[1], 0.7)  # 0.1 + 0.6
P.set(jobclass[0], jobclass[0], node[1], node[0], 0.2)
P.set(jobclass[0], jobclass[0], node[1], node[1], 0.0)

# Class 1 to Class 2 routing matrix 
P.set(jobclass[0], jobclass[1], node[0], node[0], 0.0)
P.set(jobclass[0], jobclass[1], node[0], node[1], 0.0)
P.set(jobclass[0], jobclass[1], node[1], node[0], 0.8)
P.set(jobclass[0], jobclass[1], node[1], node[1], 0.0)

# Class 2 to Class 2 routing matrix
P.set(jobclass[1], jobclass[1], node[0], node[0], 0.0)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)
P.set(jobclass[1], jobclass[1], node[1], node[0], 0.0)
P.set(jobclass[1], jobclass[1], node[1], node[1], 0.0)

# Class 2 to Class 1 routing matrix
P.set(jobclass[1], jobclass[0], node[0], node[0], 0.0) 
P.set(jobclass[1], jobclass[0], node[0], node[1], 0.0)
P.set(jobclass[1], jobclass[0], node[1], node[0], 1.0)
P.set(jobclass[1], jobclass[0], node[1], node[1], 0.0)

model.link(P)

In [3]:
# Note: Advanced transient analysis features are not yet available in Python LINE
# This example demonstrates the multi-class model setup
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.9916 0.9916 0.7937  0.7937 1.2494 1.2494
  Delay   Class2 0.4442 0.4442 0.4444  0.4444 0.9995 0.9995
 Queue1   Class1 1.1304 0.5216 0.9048  0.9048 1.2494 1.2494
 Queue1   Class2 1.0852 0.4173 1.0857  1.0857 0.9995 0.9995


In [4]:
solver_nc = NC(model)
avg_table_nc = solver_nc.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 1.6613 1.6613 1.0000  1.0000 1.6613 1.6613
  Delay   Class2 0.9303 0.9303 1.0000  1.0000 0.9303 0.9303
 Queue1   Class1 1.3380 0.3230 1.1506  1.1506 1.1629 1.1629
 Queue1   Class2 1.0704 0.3101 1.1506  1.1506 0.9303 0.9303


In [5]:
solver_jmt = JMT(model, samples=10000)
avg_table_jmt = solver_jmt.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 3.1749 3.1749 1.0007  1.0007 3.0903 3.0903
  Delay   Class2 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
 Queue1   Class1 1.8282 0.5841 0.8795  0.8795 2.1522 2.1522
 Queue1   Class2 0.0000 0.0000 0.0000  0.0000 3.7317 3.7317
